In [1]:
from sentence_transformers import SentenceTransformer # 加载和使用Embedding模型

import textwrap
import numpy as np
import pandas as pd
import os


In [2]:
DOCUMENT1 = {
    "title": "操作气候控制系统",
    "content": "您的Googlecar配备了一个气候控制系统，允许您调节车内的温度和气流。要操作气候控制系统，请使用位于中央控制台上的按钮和旋钮。\n温度：温度旋钮控制车内的温度。顺时针转动旋钮以增加温度，逆时针转动旋钮以降低温度。\n气流：气流旋钮控制车内的气流量。顺时针转动旋钮以增加气流，逆时针转动旋钮以减少气流。\n风扇速度：风扇速度旋钮控制风扇的速度。顺时针转动旋钮以增加风扇速度，逆时针转动旋钮以降低风扇速度。\n模式：模式按钮允许您选择所需的模式。可用的模式有：\n自动：汽车将自动调节温度和气流以保持舒适的水平。\n冷却：汽车将吹入冷空气。\n加热：汽车将吹入暖空气。\n除霜：汽车将吹入暖空气到挡风玻璃上以除霜。"
}

DOCUMENT2 = {
    "title": "触摸屏",
    "content": "您的Googlecar配备了一个大尺寸触摸屏显示器，提供了访问各种功能，包括导航、娱乐和气候控制。要使用触摸屏显示器，只需触摸所需图标。\n例如，您可以触摸“导航”图标以获取前往目的地的路线，或触摸“音乐”图标以播放您喜欢的歌曲。"
}

DOCUMENT3 = {
    "title": "换挡",
    "content": "您的Googlecar配备了自动变速器。要换挡，只需将换挡杆移动到所需位置。\n驻车：此位置用于停车时使用。车轮被锁定，汽车无法移动。\n倒车：此位置用于倒车。\n空挡：此位置用于在红灯或交通拥堵时停车。汽车不在档位中，除非您踩下油门踏板，否则不会移动。\n驾驶：此位置用于前进。\n低速：此位置用于在雪地或其他湿滑条件下驾驶。"
}
documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [3]:
df = pd.DataFrame(documents)
df.columns = ['Title', 'Text']


model = SentenceTransformer('../rag_app/bge-small-zh-v1.5')

In [5]:
def embed_fn(title, text):
  return model.encode(text, normalize_embeddings=True)

df['Embeddings'] = df.apply(lambda row: embed_fn(row['Title'], row['Text']), axis=1)
df

,Title,Text,Embeddings
0,操作气候控制系统,您的Googlecar配备了一个气候控制系统，允许您调节车内的温度和气流。要操作气候控制系统...,"[0.04296706, 0.071036376, -0.02249626, 0.03746..."
1,触摸屏,您的Googlecar配备了一个大尺寸触摸屏显示器，提供了访问各种功能，包括导航、娱乐和气候...,"[0.03395347, -0.013656664, 0.056943383, 0.0310..."
2,换挡,您的Googlecar配备了自动变速器。要换挡，只需将换挡杆移动到所需位置。\n驻车：此位置...,"[0.06645521, -0.00318073, 0.07671209, 0.005903..."


In [6]:
## 使用 find_best_passage 函数计算点积，然后按照从大到小的点积值对 DataFrame 进行排序，以便从数据库中检索相关段落。
def find_best_passage(query, dataframe):
  """
  Compute the distances between the query and each document in the dataframe
  using the dot product.
  """
  query_embedding = model.encode(query, normalize_embeddings=True)
  dot_products = np.dot(np.stack(dataframe['Embeddings']), query_embedding)
  idx = np.argmax(dot_products)
  return dataframe.iloc[idx]['Text'] # Return text from index with max value

In [7]:
query = "如何在谷歌汽车中换挡？"
## 查看数据库中最相关的文档：
passage = find_best_passage(query, df)
passage

'您的Googlecar配备了自动变速器。要换挡，只需将换挡杆移动到所需位置。\n驻车：此位置用于停车时使用。车轮被锁定，汽车无法移动。\n倒车：此位置用于倒车。\n空挡：此位置用于在红灯或交通拥堵时停车。汽车不在档位中，除非您踩下油门踏板，否则不会移动。\n驾驶：此位置用于前进。\n低速：此位置用于在雪地或其他湿滑条件下驾驶。'

In [8]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = textwrap.dedent("""是一个有用且信息丰富的机器人，使用下面包含的参考文本来回答问题。
请务必用完整的句子回答，内容要全面，包括所有相关的背景信息。
不过，你在与非技术观众交流，所以要确保简化复杂的概念，
并保持友好和对话的语气。
如果文本与答案无关，你可以忽略它。
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

In [9]:
prompt = make_prompt(query, passage)
print(prompt)

是一个有用且信息丰富的机器人，使用下面包含的参考文本来回答问题。
请务必用完整的句子回答，内容要全面，包括所有相关的背景信息。
不过，你在与非技术观众交流，所以要确保简化复杂的概念，
并保持友好和对话的语气。
如果文本与答案无关，你可以忽略它。
  QUESTION: '如何在谷歌汽车中换挡？'
  PASSAGE: '您的Googlecar配备了自动变速器。要换挡，只需将换挡杆移动到所需位置。 驻车：此位置用于停车时使用。车轮被锁定，汽车无法移动。 倒车：此位置用于倒车。 空挡：此位置用于在红灯或交通拥堵时停车。汽车不在档位中，除非您踩下油门踏板，否则不会移动。 驾驶：此位置用于前进。 低速：此位置用于在雪地或其他湿滑条件下驾驶。'

    ANSWER:


In [12]:
import google.generativeai as genai
from IPython.display import Markdown
from dotenv import load_dotenv
load_dotenv()

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

model = genai.GenerativeModel('gemini-1.5-pro-latest')
answer = model.generate_content(prompt)

Markdown(answer.text)

由于你的谷歌汽车使用的是自动变速器，你只需将换挡杆移动到所需的位置即可。有几个不同的位置，每个位置都有其用途：

* **驻车（P）：**当你停车时使用这个位置。它会锁住车轮，防止汽车移动。

* **倒车（R）：**当你需要倒车时使用这个位置。

* **空挡（N）：** 当你在红灯或交通拥堵时暂时停车时使用这个位置。汽车不在档位中，除非你踩油门，否则不会移动。

* **驾驶（D）：** 当你想向前行驶时使用这个位置。

* **低速（L）：** 当你在雪地或其他湿滑路面行驶时使用这个位置，它能提供更大的牵引力。
